In [1]:
import numpy as np
import scipy.stats
import scs

In [2]:
#############################################
#      Generate random cone problems        #
#############################################

def pos(x):
    return (x + np.abs(x)) / 2.
    
def gen_feasible(m, n, p_scale = 0.1):
    z = np.random.randn(m)
    y = pos(z)
    s = y - z

    P = np.random.randn(n,n)
    P = p_scale * P.T @ P

    # Make problem slightly more numerically challenging:
    A = np.random.randn(m, n)
    U, S, V = np.linalg.svd(A, full_matrices=False)
    S = S**2
    S /= np.max(S)
    A = (U * S) @ V
        
    x = np.random.randn(n)
    c = -A.T @ y - P @ x
    b = A.dot(x) + s
    
    #b /= np.linalg.norm(b)
    #x /= np.linalg.norm(b)
    
    #c /= np.linalg.norm(c)
    #y /= np.linalg.norm(c)
    
    return (P, A, b, c, x, y)

In [3]:
def gen_infeasible(m, n, p_scale = 0.1):
    # b'y < 0, A'y == 0
    z = np.random.randn(m)
    y = pos(z)  # y = s - z;
    
    A = np.random.randn(m, n)
    
    # A := A - y(A'y)' / y'y
    A = A - np.outer(y, np.transpose(A).dot(y)) / np.linalg.norm(y)**2

    b = np.random.randn(m)
    b = -b / np.dot(b, y)
    
    P = np.random.randn(n,n)
    P = p_scale * P.T @ P

    return (P, A, b, np.random.randn(n))

def gen_unbounded(m, n, p_scale = 0.1):
    # c'x < 0, Ax + s = 0, Px = 0
    z = np.random.randn(m)
    s = pos(z)
    
    P = np.random.randn(n,n)
    P = p_scale * P.T @ P
    eigs, V = np.linalg.eig(P)
    
    i = np.argmin(eigs)
    eigs[i] = 0
    x = V[:,i]
    
    # Px = 0
    P = (V * eigs) @ V.T
      
    A = np.random.randn(m, n)

    # A := A - (s + Ax)x' / x'x ===> Ax + s == 0
    A = A - np.outer(s + A.dot(x), x) / np.linalg.norm(x)**2
        
    c = np.random.randn(n)
    c = -c / np.dot(c, x)
    
    return (P, A, np.random.randn(m), c)

In [4]:
def is_optimal(P, A, b, c, x, y, tol=1e-6):
    s = b - A @ x
    
    if (np.linalg.norm(P @ x + c + A.T @ y) < tol and
        np.abs(y.T @ s) < tol and
        np.linalg.norm(s - pos(s)) < tol and
        np.linalg.norm(y - pos(y)) < tol):
        return True
    return False

def is_infeasible(A, b, y, tol=1e-6):
    if b.T @ y >= 0:
        return False
    
    y_hat = y / np.abs(b.T @ y)
    if (np.linalg.norm(y_hat - pos(y_hat)) < tol and np.linalg.norm(A.T @ y_hat) < tol):
        return True
    return False

def is_unbounded(P, A, c, x, tol=1e-6):
    if c.T @ x >= 0:
        return False
    
    x_hat = x / np.abs(c.T @ x)
    if np.linalg.norm(P @ x_hat) < tol and np.linalg.norm(A @ x_hat + pos(-A @ x_hat)) < tol:
        return True
    return False

In [5]:
# ''linear'' projection logic

class LinearProjector(object):
    def __init__(self, P, A, b, c):
        (m,n) = A.shape
        self.A = A
        self.h = np.hstack((c, b))        
        self.L = scipy.linalg.cho_factor(np.eye(n) + P + A.T @ A)
        self.g = self._solve(self.h)

    def _solve(self, v):
        (m, n) = self.A.shape
        sol = np.zeros(n+m,)
        sol[:n] = scipy.linalg.cho_solve(self.L, v[:n] - self.A.T @ v[n:])
        sol[n:] = v[n:] + self.A @ sol[:n]
        return sol
        
    def project(self, w):
        g = self.g
        p = self._solve(w[:-1])
        
        _a = 1 + g.T@g
        _b = w[:-1].T @ g - 2 * p.T @ g - w[-1]
        _c = p.T@p - w[:-1].T @ p
        
        tau = (-_b + np.sqrt(_b ** 2 - 4 * _a * _c)) / 2 / _a
        
        return np.hstack((p - tau * g, tau))

# Douglas-Rachford splitting / ADMM for QP
def solve_qp_dr(P, A, b, c, N=1000, tol=1e-6):
    (m,n) = np.shape(A)

    lp = LinearProjector(P, A, b, c)

    u = np.zeros(n+m+1,)
    #u[:n+m] = np.hstack((c, b))
    u[-1] = 1.

    def proj_cone(v):
        v[n:] = np.maximum(v[n:], 0.)
        return v


    use_dr = True # slightly different DR vs ADMM
    
    for i in range(N):
        # for DR ut converges to sol *not* u, see Patrinos, Stella, Bemporad, 2014
        # v - ut go to zero
        dr_lam = 0.5 # \in [0,1], 0.5 = DR, 1.0 = PR

        ut = lp.project(u)
        v = proj_cone(2 * ut - u)
        u += 2 * dr_lam * (v - ut)
        
        x = ut[:n] / ut[-1]
        y = ut[n:-1] / ut[-1]
        
        if (is_optimal(P, A, b, c, x, y, tol=tol) or 
            is_infeasible(A, b, y, tol=tol) or
            is_unbounded(P, A, c, x, tol=tol)):
            break
        
    print(i)
    return x,y


In [6]:
m = 150
n = 100
N = int(5e3)

In [38]:
np.random.seed(123)
(P, A, b, c) = gen_unbounded(m, n)
#P = P + 1e-7 * np.eye(n)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=10 * N, acceleration_lookback = 10, eps=1e-6)

3263
----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 15000, nnz in P = 5050
eps = 1.00e-06, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 100, constraints m = 150
Cones:	linear vars: 150
Setup time: 7.46e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.12e+18  1.46e+18  3.40e-01 -1.42e+19  1.64e+19  1.44e+19  3.89e-03 
   100| 6.85e+10  9.39e+10  1.00e+00 -2.18e+18  2.44e+11  2.97e+16  1.18e-02 
   120| 1.41e+09  2.65e+09  1.00e+00 -2.18e+18  1.66e+09  2.97e+16  1.48e-02 
-----------------------

In [8]:
print(c.T @ sol['x'])
print(np.linalg.norm(c) * np.linalg.norm(A @ sol['x'] + sol['s']))
print(np.sqrt(max(0., sol['x'].T @ P @ sol['x'])))

-1.0000000000000007
1.300535183597621e-07
0.0


In [9]:
np.random.seed(123)
(P, A, b, c) = gen_infeasible(m, n)
#P = P + np.eye(n)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6)

47
----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 15000, nnz in P = 5050
eps = 1.00e-06, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 100, constraints m = 150
Cones:	linear vars: 150
Setup time: 1.23e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 8.46e+18  4.36e+18  3.63e-01 -3.32e+19  9.41e+17  9.04e+18  9.78e-03 
    80| 2.43e+08  5.81e+07  1.00e+00 -1.49e+08  2.11e+18  3.93e+11  1.76e-02 
----------------------------------------------------------------------------
Status: Infeasible
Timing: 

In [10]:
print(b.T @ sol['y'])
print(np.linalg.norm(b) * np.linalg.norm(A.T @ sol['y']))

-1.0000000000000013
1.6939520478753857e-09


In [26]:
np.random.seed(123)
(P, A, b, c, _x, _y) = gen_feasible(m, n, p_scale=0.1)
#(_x,_y) = solve_qp_dr(P, A, b, c, N=int())
probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)

cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 5, eps=1e-9, scale=0.1)

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 15000, nnz in P = 5050
eps = 1.00e-09, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 0.10
acceleration_lookback = 5, rho_x = 1.00e-03
Variables n = 100, constraints m = 150
Cones:	linear vars: 150
Setup time: 9.55e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 5.61e+18  6.14e+18  7.26e-01 -6.63e+21  1.50e+20  2.96e+21  3.94e-03 
   100| 3.59e-05  1.26e-05  1.29e-06 -1.11e+03 -3.03e+00  0.00e+00  1.07e-02 
   200| 3.07e-06  1.71e-06  8.32e-07 -1.11e+03 -3.12e+00  0.00e+00  1.74e-02 
   300| 3.32e-07  8.94e-08  1.

In [27]:
x = sol['x']
y = sol['y']
s = sol['s']

print(np.linalg.norm(x - _x))
print(np.linalg.norm(y - _y))

#x = _x
#y = _y
#s = b - A @ _x

print(np.linalg.norm(A @ x + s - b) / (1+np.linalg.norm(b)))
print(np.linalg.norm(P@x + A.T @ y + c) / (1+np.linalg.norm(c)))
print(abs(x.T@ P @ x + c.T @ x + b.T @ y) / (1 + abs(x.T@P @ x) + abs(c.T @ x) + abs(b.T @ y)))

1.7281614908610811e-07
5.968280239206964e-05
7.400450858741908e-10
3.7900877077690046e-10
1.7173181966403902e-10


In [13]:
np.random.seed(123)
(P, A, b, c, _, _) = gen_feasible(m, n, p_scale=0.)
#(x,y) = solve_qp_dr(P, A, b, c, N=N)
#print(c.T @ x)
#print(x)

In [14]:
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 0, eps=1e-6, use_indirect=False)
print(c.T @ sol['x'])

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 15000, nnz in P = 5050
eps = 1.00e-06, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 0, rho_x = 1.00e-03
Variables n = 100, constraints m = 150
Cones:	linear vars: 150
Setup time: 7.10e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 9.25e+16  1.66e+17  7.13e-01 -1.17e+20  2.03e+19  4.85e+20  3.78e-03 
   100| 4.01e-06  2.85e-06  3.56e-07 -1.11e+03 -3.07e+00  0.00e+00  8.83e-03 
   200| 1.00e-06  3.66e-07  1.69e-08 -1.11e+03 -3.11e+00  0.00e+00  1.41e-02 
   220| 8.07e-07  2.82e-07  1.

In [15]:
probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 0, eps=1e-, use_indirect=True)
print(c.T @ sol['x'])

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 15000, nnz in P = 0
	 cg_rate = 2.00
eps = 1.00e-06, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 0, rho_x = 1.00e-03
Variables n = 100, constraints m = 150
Cones:	linear vars: 150
Setup time: 1.78e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.82e+17  1.42e+17  8.33e-01 -1.61e+19 -1.47e+18  9.99e+18  2.35e-02 
   100| 1.20e-03  1.33e-03  1.28e-03 -3.15e+00 -3.14e+00  0.00e+00  3.34e-01 
   200| 4.61e-04  2.18e-04  1.44e-05 -3.13e+00 -3.13e+00  0.00e+00  6.14e-01 
   300| 1.97e-

In [16]:
np.random.seed(123)
(P, A, b, c, _, _) = gen_feasible(m, n, 0.001)

In [17]:
print(np.linalg.norm(A.flatten(), np.inf))
print(np.linalg.norm(P.flatten(), np.inf))

0.1348910898626595
0.13052636197202544


In [18]:
(x,y) = solve_qp_dr(P, A, b, c, N=N)
print(x)
print(y)

2155
[ 0.63896206 -0.34713304  0.55473289 -0.83507225  2.47242503  0.90257704
  0.14915616  0.34890192  0.79075734 -0.51177026 -0.55692527  0.52206908
  0.31072125  0.59577785  1.79530395  1.11765212 -0.23454035 -1.36534556
 -0.97981499 -0.84848097  0.72061371 -0.37665119 -0.85770455 -1.42430804
 -0.01881189 -0.69399211 -1.05429813  1.38319776 -0.88005758 -0.24018333
 -0.96484521  1.43855978 -0.6761082  -0.58423952 -1.15288591 -1.23040016
 -0.63208419 -0.22276001 -1.22994142  0.56559664 -1.21310959  0.73552419
  0.38294666 -0.23143657  0.12149417 -0.26075266  1.00679611 -1.48670435
 -0.97189256 -1.65550318 -1.14048947  0.49224182 -0.47702848 -2.4372688
  1.86907522 -1.17440664  0.62306335  0.77449313 -0.58113997 -0.63049732
 -0.2146924  -0.3478203  -2.41506909 -0.37101928  1.71428078  1.39083162
 -0.63965172  0.40814185  1.2308312   0.45607446 -0.9512896  -0.59060914
  1.24532828 -1.34477912  0.90352266  1.57904241 -0.87583685 -0.24298466
  0.76234115 -0.72030367 -0.37551384 -0.3043628

In [19]:
np.random.seed(123)
(P, A, b, c, _, _) = gen_feasible(m, n)

probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)

cone = dict(l=m)
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 5, eps=1e-6, scale=1)

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 15000, nnz in P = 5050
eps = 1.00e-06, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 5, rho_x = 1.00e-03
Variables n = 100, constraints m = 150
Cones:	linear vars: 150
Setup time: 7.79e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 9.25e+16  1.66e+17  7.13e-01 -1.17e+20  2.03e+19  4.85e+20  3.14e-03 
   100| 2.01e-06  2.28e-06  1.14e-06 -1.11e+03 -3.13e+00  0.00e+00  9.59e-03 
   120| 4.33e-07  6.36e-07  2.49e-07 -1.11e+03 -3.13e+00  0.00e+00  1.13e-02 
------------------------------

{'x': array([ 0.63895157, -0.34712354,  0.55471909, -0.83508383,  2.47243298,
         0.90257049,  0.14913651,  0.34889258,  0.79073715, -0.51176192,
        -0.55692743,  0.52207556,  0.31073184,  0.59578373,  1.7952948 ,
         1.11764931, -0.23453912, -1.36534058, -0.97980997, -0.8484768 ,
         0.72060373, -0.37665023, -0.85770712, -1.42430012, -0.01881832,
        -0.6939973 , -1.05428352,  1.38318239, -0.88004463, -0.24017243,
        -0.96485454,  1.43855282, -0.67611053, -0.58424109, -1.15289871,
        -1.23038234, -0.63209177, -0.22274636, -1.22992516,  0.56559298,
        -1.21310522,  0.73551561,  0.38293886, -0.23143547,  0.12150647,
        -0.26074384,  1.00680208, -1.48669857, -0.97189493, -1.65550716,
        -1.14047747,  0.49224195, -0.47702618, -2.43726565,  1.86907659,
        -1.1744143 ,  0.62308288,  0.77446694, -0.58115046, -0.63048575,
        -0.21470531, -0.34781554, -2.41504692, -0.37101958,  1.71428125,
         1.39083757, -0.63965009,  0.40813034,

In [20]:
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 5, eps=1e-6, use_indirect=True, scale=1)

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 15000, nnz in P = 5050
	 cg_rate = 2.00
eps = 1.00e-06, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 5, rho_x = 1.00e-03
Variables n = 100, constraints m = 150
Cones:	linear vars: 150
Setup time: 6.78e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 9.25e+16  1.66e+17  7.13e-01 -1.17e+20  2.03e+19  4.85e+20  6.99e-03 
   100| 4.39e-07  8.33e-07  2.30e-07 -1.11e+03 -3.13e+00  0.00e+00  1.03e-01 
----------------------------------------------------------------------------
Status: Solv

{'x': array([ 0.63895036, -0.3471272 ,  0.55471999, -0.83508205,  2.4724326 ,
         0.90257559,  0.14913895,  0.34889451,  0.79074172, -0.51176536,
        -0.55692868,  0.52207229,  0.31072857,  0.59578396,  1.79529801,
         1.11764782, -0.23453962, -1.36534222, -0.97981125, -0.84848028,
         0.72060727, -0.3766536 , -0.85771156, -1.4243063 , -0.01881561,
        -0.69399389, -1.054291  ,  1.38318609, -0.88004745, -0.24017028,
        -0.96485674,  1.43855419, -0.67610853, -0.58424304, -1.15289768,
        -1.23038182, -0.63209006, -0.22274723, -1.22993124,  0.56559548,
        -1.21310392,  0.73551865,  0.38294563, -0.23143733,  0.12150525,
        -0.26074806,  1.00679945, -1.48669807, -0.9719007 , -1.65550416,
        -1.14047998,  0.49224534, -0.47702303, -2.43727013,  1.86907863,
        -1.1744082 ,  0.62307905,  0.7744716 , -0.58114608, -0.63049197,
        -0.21469908, -0.34781757, -2.41505341, -0.37101711,  1.71428178,
         1.39083401, -0.63964748,  0.40813043,

In [21]:
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 0, eps=1e-6, use_indirect=True, scale=1)

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 15000, nnz in P = 5050
	 cg_rate = 2.00
eps = 1.00e-06, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 0, rho_x = 1.00e-03
Variables n = 100, constraints m = 150
Cones:	linear vars: 150
Setup time: 6.13e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 9.25e+16  1.66e+17  7.13e-01 -1.17e+20  2.03e+19  4.85e+20  6.70e-03 
   100| 4.01e-06  2.84e-06  3.63e-07 -1.11e+03 -3.07e+00  0.00e+00  9.36e-02 
   200| 1.00e-06  3.75e-07  9.10e-09 -1.11e+03 -3.11e+00  0.00e+00  1.17e-01 
   220| 8.0

{'x': array([ 0.63899764, -0.34717676,  0.5547943 , -0.83503866,  2.47241534,
         0.90258705,  0.14921113,  0.34894626,  0.79085766, -0.5118175 ,
        -0.55691303,  0.52205062,  0.3107039 ,  0.59574086,  1.79532763,
         1.11766924, -0.23455334, -1.3653598 , -0.97983477, -0.84849874,
         0.72065993, -0.37665125, -0.8576711 , -1.42433303, -0.0188106 ,
        -0.69397966, -1.05434534,  1.38325103, -0.8801115 , -0.24023111,
        -0.96480709,  1.43859661, -0.6760924 , -0.58425013, -1.15283868,
        -1.23047433, -0.63205487, -0.22282078, -1.22999608,  0.56560604,
        -1.21312845,  0.73559207,  0.38298591, -0.23143844,  0.12141887,
        -0.26076504,  1.00678147, -1.48673277, -0.97188211, -1.65549102,
        -1.14055854,  0.49225721, -0.47705548, -2.43727638,  1.86905562,
        -1.17437639,  0.62298557,  0.77459177, -0.58111773, -0.63052411,
        -0.2146475 , -0.34783991, -2.41514526, -0.37102154,  1.71427863,
         1.39082508, -0.63966717,  0.40819097,

In [22]:
np.random.seed(123)
(P, A, b, c, _, _) = gen_feasible(m, n, p_scale=0.)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
print(x)

probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6)

probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6)

4999
[ 0.20545814 -0.05100439  0.61618934 -0.83025321  1.8768222   1.26621898
 -0.41457764 -0.05651394  0.56233689 -0.32514571 -0.01103386  0.2258596
  0.0694998   0.07767681  1.46315403  0.84042595 -0.04570704 -1.3807189
 -0.88943627 -1.12706108  0.80276804 -0.41535013 -0.80798069 -1.71900688
  0.21907532 -0.91731893 -1.8298685   1.49025353 -0.49911     0.05271534
 -1.1418378   1.38571157  0.27360366 -0.71218066 -1.47143771 -1.03568865
 -1.01523352  0.38106151 -1.38092843  1.10909274 -1.09238166  0.52079945
  0.08850571 -0.28529819  0.13645798  0.14597993  0.09100091 -1.19965069
 -1.65094725 -0.95101286 -0.32600079  0.15588202 -0.67543839 -2.39784109
  1.5510626  -1.13859854  0.45306934  0.41010498 -0.09703006 -0.71934473
 -0.53443063 -0.64951018 -1.62921329 -0.85850511  1.22877619  1.07409282
 -0.36909815  0.22302632  1.5261399   0.12242534  0.06083797 -0.58260368
  1.27101538 -1.46318896  0.66199798  1.35180975 -0.80145505  0.10047618
  0.99456596 -0.38510643 -0.67805403 -0.64468307

{'x': array([ 1.36597052e-01, -4.80191612e-02, -9.04391800e+00, -1.08803451e+00,
         8.02950589e+00, -2.67297795e+00, -1.82150350e+00,  3.49606120e+00,
        -3.04654002e+00,  1.99131578e+00, -3.42243261e+00,  5.26113551e+00,
        -1.24583106e+00,  9.24124730e+00,  8.07373332e+00,  3.45106778e-01,
        -6.45616686e+00, -2.89676818e+00,  7.98928310e-01,  7.60906554e+00,
         9.27717776e+00,  5.80143051e+00, -2.69975509e+00, -4.94158968e+00,
         7.54810272e+00, -5.46199009e+00,  3.03696882e+00,  9.43383689e+00,
        -2.53017435e+00,  5.16789108e+00, -3.69396587e+00,  3.67464299e+00,
        -2.85601208e+00,  3.45525020e+00, -1.21531898e+01, -5.15838619e+00,
         4.64391428e+00, -5.76074758e+00,  3.32791526e+00, -4.61815976e+00,
        -7.64584085e-01, -1.71935991e+00,  2.82520475e+00, -1.84207076e+00,
         4.87360541e+00, -5.02118577e+00,  1.70781070e+00, -5.79912483e+00,
         9.05778713e-01, -5.49143709e-01, -8.25797273e+00, -3.12154325e+00,
       

In [23]:
sol = scs.solve(probdata, cone, normalize=False, max_iters=N, acceleration_lookback = 0, eps=1e-6, use_indirect=True)
print(c.T @ sol['x'])

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 15000, nnz in P = 0
	 cg_rate = 2.00
eps = 1.00e-06, alpha = 1.50, max_iters = 5000, normalize = 0, scale = 1.00
acceleration_lookback = 0, rho_x = 1.00e-03
Variables n = 100, constraints m = 150
Cones:	linear vars: 150
Setup time: 3.91e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 4.26e+17  2.38e+17  8.23e-01 -2.10e+19 -2.03e+18  1.55e+19  8.36e-03 
   100| 2.45e-03  1.60e-03  1.01e-04 -3.13e+00 -3.13e+00  0.00e+00  1.83e-01 
   200| 7.42e-04  3.87e-04  2.69e-04 -3.14e+00 -3.13e+00  0.00e+00  3.52e-01 
   300| 2.59e-

In [24]:
probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6, use_indirect=True)
print(c.T @ sol['x'])

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 15000, nnz in P = 0
	 cg_rate = 2.00
eps = 1.00e-06, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 100, constraints m = 150
Cones:	linear vars: 150
Setup time: 1.27e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.82e+17  1.42e+17  8.33e-01 -1.61e+19 -1.47e+18  9.99e+18  1.74e-02 
   100| 1.84e-03  7.94e-04  2.75e-04 -3.13e+00 -3.13e+00  0.00e+00  4.38e-01 
   200| 9.16e-04  4.58e-04  5.22e-04 -3.13e+00 -3.13e+00  0.00e+00  8.93e-01 
   300| 3.38e

In [25]:
np.random.seed(123)
(P, A, b, c, _, _) = gen_feasible(m, n, p_scale=0.)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
print(c.T @ x)
print(x)

probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6, use_indirect=True)

probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6, use_indirect=True)
print(c.T @ sol['x'])

4999
-3.1335340624620724
[ 0.20545814 -0.05100439  0.61618934 -0.83025321  1.8768222   1.26621898
 -0.41457764 -0.05651394  0.56233689 -0.32514571 -0.01103386  0.2258596
  0.0694998   0.07767681  1.46315403  0.84042595 -0.04570704 -1.3807189
 -0.88943627 -1.12706108  0.80276804 -0.41535013 -0.80798069 -1.71900688
  0.21907532 -0.91731893 -1.8298685   1.49025353 -0.49911     0.05271534
 -1.1418378   1.38571157  0.27360366 -0.71218066 -1.47143771 -1.03568865
 -1.01523352  0.38106151 -1.38092843  1.10909274 -1.09238166  0.52079945
  0.08850571 -0.28529819  0.13645798  0.14597993  0.09100091 -1.19965069
 -1.65094725 -0.95101286 -0.32600079  0.15588202 -0.67543839 -2.39784109
  1.5510626  -1.13859854  0.45306934  0.41010498 -0.09703006 -0.71934473
 -0.53443063 -0.64951018 -1.62921329 -0.85850511  1.22877619  1.07409282
 -0.36909815  0.22302632  1.5261399   0.12242534  0.06083797 -0.58260368
  1.27101538 -1.46318896  0.66199798  1.35180975 -0.80145505  0.10047618
  0.99456596 -0.38510643 -0.